In [1]:
!pip install qiskit qiskit-machine-learning --upgrade


In [2]:
pip install qiskit-aer

  Using cached qiskit-aer-0.15.1.tar.gz (6.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for qiskit-aer (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [126 lines of output]
      
      
      --------------------------------------------------------------------------------
      -- Trying 'Ninja' generator
      --------------------------------
      ---------------------------
      ----------------------
      -----------------
      ------------
      -------
      --
      Not searching for unused variables given on the command line.
      -- The C compiler identification is AppleClang 16.0.0.16000026
      -- Detecting C compiler ABI info
      -- Detecting C compiler ABI info - done
      -- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc

In [3]:
# Ensure the required packages are installed
!pip install qiskit qiskit-machine-learning



In [8]:
# Ensure the required packages are installed
!pip install qiskit --upgrade




In [5]:
!pip install qiskit --upgrade


In [6]:
!pip install qiskit
!pip install qiskit-nature
!pip install 'qiskit[visualization]'
!pip install qiskit.utils
!pip install pyscf

In [9]:
from qiskit.visualization import array_to_latex, plot_bloch_vector, plot_bloch_multivector, plot_state_qsphere, plot_state_city
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator, Aer

from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer
from qiskit_nature import settings
from qiskit.primitives import Sampler, StatevectorSampler
from qiskit_algorithms import HamiltonianPhaseEstimation, PhaseEstimation
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import SLSQP, SPSA, QNSPSA
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit.circuit.library import TwoLocal
from functools import partial as apply_variation
from qiskit.circuit.library import Initialize
from qiskit.quantum_info import SparsePauliOp
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.operators import ElectronicIntegrals
from pyscf import gto, scf


ModuleNotFoundError: No module named 'qiskit_aer'

In [10]:
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from qiskit import QuantumCircuit
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram

# 1. Load and extract features from CAN messages
def load_can_messages(directory):
    can_messages = []
    files = sorted([f for f in os.listdir(directory) if f.endswith('.bin')])
    for file_name in files[:3]:  # Load a limited number of files for memory efficiency
        file_path = os.path.join(directory, file_name)
        with open(file_path, 'rb') as f:
            messages = np.frombuffer(f.read(), dtype=np.uint8)
            can_messages.append(messages)
    return can_messages

def extract_temporal_features(messages):
    intervals = []
    message_counts = []
    for msg_set in messages:
        timestamps = msg_set[::2]  # Assuming every 2 bytes represent a timestamp or interval
        diffs = np.diff(timestamps)
        intervals.append(diffs)
        message_counts.append(len(msg_set))
    return intervals, message_counts

# 2. Normalize and prepare data
def prepare_data(intervals, message_counts):
    features = np.array([np.mean(interval) for interval in intervals if len(interval) > 0])  # Average interval time
    counts = np.array(message_counts)
    data = np.vstack((features, counts)).T
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    return data

# 3. Build a simple Quantum Circuit (QNN-like structure)
def create_quantum_circuit(data):
    num_features = data.shape[1]
    quantum_circuits = []
    for i in range(data.shape[0]):
        qc = QuantumCircuit(num_features)
        for j in range(num_features):
            qc.ry(data[i, j], j)  # Encode each feature as a rotation angle
        quantum_circuits.append(qc)
    return quantum_circuits

if __name__ == "__main__":
    # Dataset directory for Google Colab
    dataset_directory = '/content/datasets'

    # Load data
    can_messages = load_can_messages(dataset_directory)
    print("Loaded CAN message sets:", len(can_messages))

    # Extract features (intervals and counts) and prepare data
    intervals, message_counts = extract_temporal_features(can_messages)
    data = prepare_data(intervals, message_counts)

    # Build quantum circuits (without machine learning component)
    quantum_circuits = create_quantum_circuit(data)

    print("Created quantum circuits for the dataset.")


    # Import necessary libraries from Qiskit
from qiskit import QuantumCircuit
from qiskit_aer import Aer, AerError
from qiskit.visualization import plot_histogram

# Function to create a quantum circuit for a given feature vector
def create_quantum_circuit(features):
    n = len(features)  # Number of features
    circuit = QuantumCircuit(n)  # Create a quantum circuit with 'n' qubits

    # Encode features into the quantum circuit (example with RY gates)
    for i in range(n):
        circuit.ry(features[i], i)  # Rotate around Y-axis with feature value

    # Add measurement to all qubits
    circuit.measure_all()
    return circuit

# Function to run the quantum circuit on a classical simulator
def run_quantum_circuit(circuit):
    backend = Aer.get_backend('qasm_simulator')  # Use the QASM simulator
    # Run the quantum circuit on the simulator
    job = backend.run(circuit, shots=1000)  # Run the circuit
    result = job.result()  # Get the result
    counts = result.get_counts(circuit)  # Get counts of results
    return counts

# Example usage
feature_data = [[0.5, 0.1, 0.3], [0.9, 0.8, 0.1]]
created_circuits = [create_quantum_circuit(features) for features in feature_data]

# Run each circuit and collect counts
all_counts = [run_quantum_circuit(circuit) for circuit in created_circuits]
# Convert binary predictions to labels (assuming 2 outcomes: 0 and 1)
label_map = {
        '0': 'normal',
        '1': 'attack',}
predicted_labels = [max(counts, key=counts.get) for counts in all_counts]
predicted_labels = [label_map.get(pred, 'normal') for pred in predicted_labels]

print("Predicted labels:", predicted_labels)



Matplotlib is building the font cache; this may take a moment.


FileNotFoundError: [Errno 2] No such file or directory: '/content/datasets'

In [11]:
    # Display histograms and predicted labels
    for i, counts in enumerate(all_counts):
        print(f"Counts for circuit {i+1}: {counts}")  # Print counts for debugging

        # Create and display the histogram
        fig, ax = plt.subplots(2, 1, figsize=(8, 10))  # Two rows: one for histogram, one for predicted labels
        plot_histogram(counts, ax=ax[0])  # Plot histogram on the first axes
        ax[0].set_title(f'Counts for circuit {i + 1}')  # Title for histogram

        # Plot predicted labels
        ax[1].bar(['normal', 'attack'], [predicted_labels.count('normal'), predicted_labels.count('attack')], color=['blue', 'red'])
        ax[1].set_title('Predicted Labels Count')
        ax[1].set_ylabel('Count')
        ax[1].set_xlabel('Predicted Label')

        plt.tight_layout()  # Adjust layout for better spacing
        plt.show()  # Show both plots

NameError: name 'all_counts' is not defined